# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 15 2022 10:34AM,253175,10,CTF0011544,"Citieffe, Inc.",Released
1,Dec 15 2022 10:29AM,253173,10,Enova-11104,Emerginnova,Released
2,Dec 15 2022 10:29AM,253173,10,Enova-11105,Emerginnova,Released
3,Dec 15 2022 10:27AM,253170,10,PRF-44712,Bio-PRF,Released
4,Dec 15 2022 10:27AM,253170,10,PRF-44715,Bio-PRF,Released
5,Dec 15 2022 10:27AM,253170,10,PRF-44722,Bio-PRF,Released
6,Dec 15 2022 10:27AM,253170,10,PRF-44727,Bio-PRF,Released
7,Dec 15 2022 10:27AM,253170,10,PRF-44734,Bio-PRF,Released
8,Dec 15 2022 10:27AM,253170,10,PRF-44738,Bio-PRF,Released
9,Dec 15 2022 10:27AM,253170,10,PRF-44742,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,253170,Released,20
38,253171,Executing,1
39,253172,Released,1
40,253173,Released,2
41,253175,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
253170,NaN,NaN,NaN,20.0
253171,NaN,NaN,1.0,NaN
253172,NaN,NaN,NaN,1.0
253173,NaN,NaN,NaN,2.0
253175,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
253064,0.0,0.0,0.0,22.0
253087,0.0,11.0,12.0,2.0
253089,0.0,0.0,1.0,0.0
253090,0.0,0.0,0.0,43.0
253091,0.0,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
253064,0.0,0,0,22
253087,0.0,11,12,2
253089,0.0,0,1,0
253090,0.0,0,0,43
253091,0.0,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,253064,0.0,0,0,22
1,253087,0.0,11,12,2
2,253089,0.0,0,1,0
3,253090,0.0,0,0,43
4,253091,0.0,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,253064,0.0,,,22
1,253087,0.0,11,12,2
2,253089,0.0,,1,
3,253090,0.0,,,43
4,253091,0.0,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 15 2022 10:34AM,253175,10,"Citieffe, Inc."
1,Dec 15 2022 10:29AM,253173,10,Emerginnova
3,Dec 15 2022 10:27AM,253170,10,Bio-PRF
23,Dec 15 2022 10:26AM,253172,22,"NBTY Global, Inc."
24,Dec 15 2022 10:25AM,253171,22,"NBTY Global, Inc."
25,Dec 15 2022 10:18AM,253169,10,ISDIN Corporation
26,Dec 15 2022 10:15AM,253168,10,"Methapharm, Inc."
38,Dec 15 2022 10:15AM,253168,10,Methapharm-G
43,Dec 15 2022 9:49AM,253167,19,"GUSA Granules USA, Inc."
44,Dec 15 2022 9:26AM,253166,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Dec 15 2022 10:34AM,253175,10,"Citieffe, Inc.",0.0,,,1
1,Dec 15 2022 10:29AM,253173,10,Emerginnova,0.0,,,2
2,Dec 15 2022 10:27AM,253170,10,Bio-PRF,0.0,,,20
3,Dec 15 2022 10:26AM,253172,22,"NBTY Global, Inc.",0.0,,,1
4,Dec 15 2022 10:25AM,253171,22,"NBTY Global, Inc.",0.0,,1,
5,Dec 15 2022 10:18AM,253169,10,ISDIN Corporation,0.0,,,1
6,Dec 15 2022 10:15AM,253168,10,"Methapharm, Inc.",0.0,,,17
7,Dec 15 2022 10:15AM,253168,10,Methapharm-G,0.0,,,17
8,Dec 15 2022 9:49AM,253167,19,"GUSA Granules USA, Inc.",0.0,,,1
9,Dec 15 2022 9:26AM,253166,10,ISDIN Corporation,0.0,,,11


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 10:34AM,253175,10,"Citieffe, Inc.",1,,
1,Dec 15 2022 10:29AM,253173,10,Emerginnova,2,,
2,Dec 15 2022 10:27AM,253170,10,Bio-PRF,20,,
3,Dec 15 2022 10:26AM,253172,22,"NBTY Global, Inc.",1,,
4,Dec 15 2022 10:25AM,253171,22,"NBTY Global, Inc.",,1,
5,Dec 15 2022 10:18AM,253169,10,ISDIN Corporation,1,,
6,Dec 15 2022 10:15AM,253168,10,"Methapharm, Inc.",17,,
7,Dec 15 2022 10:15AM,253168,10,Methapharm-G,17,,
8,Dec 15 2022 9:49AM,253167,19,"GUSA Granules USA, Inc.",1,,
9,Dec 15 2022 9:26AM,253166,10,ISDIN Corporation,11,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 10:34AM,253175,10,"Citieffe, Inc.",1,,
1,Dec 15 2022 10:29AM,253173,10,Emerginnova,2,,
2,Dec 15 2022 10:27AM,253170,10,Bio-PRF,20,,
3,Dec 15 2022 10:26AM,253172,22,"NBTY Global, Inc.",1,,
4,Dec 15 2022 10:25AM,253171,22,"NBTY Global, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 10:34AM,253175,10,"Citieffe, Inc.",1.0,NaN,NaN
1,Dec 15 2022 10:29AM,253173,10,Emerginnova,2.0,NaN,NaN
2,Dec 15 2022 10:27AM,253170,10,Bio-PRF,20.0,NaN,NaN
3,Dec 15 2022 10:26AM,253172,22,"NBTY Global, Inc.",1.0,NaN,NaN
4,Dec 15 2022 10:25AM,253171,22,"NBTY Global, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 15 2022 10:34AM,253175,10,"Citieffe, Inc.",1.0,0.0,0.0
1,Dec 15 2022 10:29AM,253173,10,Emerginnova,2.0,0.0,0.0
2,Dec 15 2022 10:27AM,253170,10,Bio-PRF,20.0,0.0,0.0
3,Dec 15 2022 10:26AM,253172,22,"NBTY Global, Inc.",1.0,0.0,0.0
4,Dec 15 2022 10:25AM,253171,22,"NBTY Global, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4050379,232.0,18.0,11.0
15,1012405,47.0,0.0,0.0
16,253125,1.0,0.0,0.0
19,2025118,39.0,15.0,8.0
20,506228,1.0,1.0,0.0
21,253132,0.0,1.0,0.0
22,506343,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4050379,232.0,18.0,11.0
1,15,1012405,47.0,0.0,0.0
2,16,253125,1.0,0.0,0.0
3,19,2025118,39.0,15.0,8.0
4,20,506228,1.0,1.0,0.0
5,21,253132,0.0,1.0,0.0
6,22,506343,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,232.0,18.0,11.0
1,15,47.0,0.0,0.0
2,16,1.0,0.0,0.0
3,19,39.0,15.0,8.0
4,20,1.0,1.0,0.0
5,21,0.0,1.0,0.0
6,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,232.0
1,15,Released,47.0
2,16,Released,1.0
3,19,Released,39.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,11.0,0.0,0.0,8.0,0.0,0.0,0.0
Executing,18.0,0.0,0.0,15.0,1.0,1.0,1.0
Released,232.0,47.0,1.0,39.0,1.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,11.0,0.0,0.0,8.0,0.0,0.0,0.0
1,Executing,18.0,0.0,0.0,15.0,1.0,1.0,1.0
2,Released,232.0,47.0,1.0,39.0,1.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,11.0,0.0,0.0,8.0,0.0,0.0,0.0
1,Executing,18.0,0.0,0.0,15.0,1.0,1.0,1.0
2,Released,232.0,47.0,1.0,39.0,1.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()